# Telegram

En este video-notebook vamos a aprender lo básico para poder hacer un bot de telegram usando Julia.

Un bot es un programa asociado a una cuenta, que permite interacción entre usuarios de telegram y el código. Funciona de 2 formas. El usuario puede enviar mensajes que el programa interpreta y el programa puede enviar mensajes al usuario. 


## Generar un bot en Telegram. 

El primer paso es generar un bot en Telegram. Por favor, mientras sean bots de prueba utilicen nombres largos y que nadie más quiera usar, como incluyendo en el nombre del bot la fecha y la versión. Después de hacer las pruebas, eliminen el bot, para no ocupar nombres y espacio de Telegram inútilmente. **Es un servicio gratuito y abierto, así que no hay que abusar de él**. 

Para generar el bot tienen que entrar a Telegram y buscar el bot, @BotFather. Lo inician (como la mayoría) con /start. 

Después agregan el comando \newbot. Les pedirá dar un nombre. Debe terminar con "bot" el nombre del bot. **Listo!!** ya crearon su bot. 

## Obteniendo el token

El token es una referencia encriptada para que telegram reconozca la interacción con el programa como legal. Su token es su llave para acceder a su bot, por lo tanto, manténgalo privado. 

Para obtener el token, dentro del mismo @BotFather, escriben, una vez creado su bot, el comando "/token". Les pedirá que seleccionen uno de sus bots. Escriban "@nombre_de_su_bot". **Listo!** @BotFather les enviará como mensaje el token. Copienlo, lo necesitarán dentro de su código. 

## Paquetería necesaria

Para interactuar entre Julia y su bot, necesitarán la paquetería Telegram. Instálenla y cargenla!

Además, carguen la paquetería Telegram.API, que les servirá para enviar mensajes de su bot al chat que seleccionen. 

In [1]:
using Telegram, Telegram.API

### id de un chat 

En muchos casos querrán que su bot sirva sólo para ustedes. Por ejemplo, que les envíe resultados parciales de una simulación mientras esta está corriendo y que sólo ustedes puedan "re-enviar" a correr alguna cosa. Para esto necesitan conocer el id del chat que tengan entre ustedes y el bot. 

Para esto necesitarán hacer su primer programa para interactuar con el bot. Utilizarán la función run_bot() incluida en la paquetería Telegram y que será la función principal para correr nuestros bots: 

In [ ]:
token = ""
tg = TelegramClient(token)
run_bot() do msg
    #chat_id = msg.message.chat.id
    @show msg
end


Las primeras 2 lineas del código son para conectar nuestro código de Julia con el bot correcto (usando el token). 

run_bot es una función que tiene como argumento una función. Lo que hace es tomar esa función y aplicársela a los mensajes que vayan generándose en el chat. 

Ahora bien, los mensajes del chat contienen mucha información además del mensaje en sí. Entre otras cosas, cada mensaje contiene  de qué chat proviene, es decir, el id del chat. 

el operador do, lo que hace es crear una función anónima que pone como primer argumento de la función que esté justo antes de do. Por ejemplo: 


In [ ]:
f(g,x) = g(x) 

f(6) do x

end


Aquí f es una función que tiene como argumentos g, una función cualquiera y x, un valor que se puede aplicar a g. Entonces, do en este caso es equivalente a f(x->x^2, 5). 

En general, se utiliza do, cuando la función anónima es más o menos complicada y cuesta entonces trabajo escribirla como x-> algo(x). Es similar a la diferencia entre definir una función como f(x) = algo o 

    function f(x)
        algo
     end
     
En el caso aplicado a run_bot, lo que se está haciendo en la función anónima es obtener el id del chat y luego mostrar este en pantalla. 

También pueden pedirle que imprima todo el mensaje "msg" y así ven qué es lo que está dentro de cada mensaje. Por supuesto, varía dependiendo del tipo de objeto que mandemos. Será diferente si enviamos una foto, un video, un documento, o texto. Incluso varía si la foto es incrustada (comprimida) o no. 

Con la función run_bot, aprendimos ya a recibir mensajes. Notarán que el texto es lo más sencillo de trabajar. msg.message.text es la cadena de caracteres con la que podemos trabajar. 

In [ ]:
run_bot() do msg
    try
        @show chat_id = msg.message.chat.id
        println(msg.message.text)
    catch
        println(msg)
    end
end

Ya sabemos cómo transformar las cadenas de caracteres en números y cómo trabajar con cadenas en sí, así que ya podemos hacer bastante. Pero ¿qué tal que ustedes quieren enviar algunas lineas de código?, en particular es común que lo que se quiera es enviar el nombre de una función junto con la variable sus argumentos. 

Una alternativa es definir un if, si la cadena de caracteres es una determinada, entonces se piden los argumentos de la función correspondiente y en orden y se corre dicha función. Esa es una forma "segura" de su bot, pero tediosa. Una alternativa es utilizar meta-programación. 

### Meta programación. 

La meta programación es de alguna forma hacer programas que hagan programas. Para esto necesitamos transformar texto en código. La función que hace esto se llama Meta.parse. Por ejemplo: 

In [ ]:
metaprograma = Meta.parse("f(x) = x^2")

In [ ]:
typeof(metaprograma)

Aquí se generó una expresión que contiene todo lo del texto. Esa expresión es una pieza de código. El siguiente paso es evaluar la expresión. Para eso utilizamos la función eval: 

In [ ]:
eval(metaprograma)

In [ ]:
f(2.3)

con esto tenemos el programa completo. Esto es útil por ejemplo debe aplicar un determinado número de ciclos for, o un conjunto de if's similares, etc. Sin embargo, hay que tener en cuenta que la meta-programación es bastante más lenta, por lo que en general es mejor evitarlo. 

Hay unas pocas excepciones. Los macros (lo que inicia con @) y cuando se usa telegram son los ejemplos principales. Así, se puede por ejemplo usar

eval(Meta.parse(msg.messag.text)) 

cuando el mensaje de texto comienza con el nombre de alguna función que hayan definido en su código, permitiendo a los usuarios aplicar algunas funciones con sus respectivos argumentos. 

El caso más simple es aplicar este truco a cualquier mensaje (en cuyo caso se vuelve simplemente un compilador de Julia, como el bot usado para aclarar dudas en el curso. Para evitar errores, uno puede utilizar try and catch. 

In [ ]:
run_bot() do msg
    try
        x = eval(Meta.parse(msg.message.text))
        println(msg.message.text)
        println(x)
    catch

    end
end

In [ ]:
f(32)

Ahora el problema es ¿cómo hacemos que el bot envíen mensajes? 

### Funciones sendMessage, sendAudio,  sendDocument, sendPhoto, sendAnimation, sendVideo

Con este propósito usaremos estas funciones. Todas requieren 2 argumentos, el chat_id y lo que se enviará. Por ejemplo sendMessage pide el karg text que es literalmente el texto que queremos enviar de regreso. 

Sobre el chat_id, pueden usar el id de su chat, o si quieren que funcione con cualquier chat, pueden poner:

chat_id = msg.message.chat.id

Hagamos un ejemplo sencillo. Por ejemplo, hagamos que nuestro bot regrese $x^2$ cuando lo que reciba como mensaje sea un número: 

In [ ]:
token = " "
tg = TelegramClient(token)

run_bot() do msg
    try
        mensaje = msg.message.text
        numeros = findall(x-> x in ['0','1','2','3','4','5','6','7','8','9'], mensaje)
        punto = findall(x-> x == '.', mensaje)
        if length(numeros)+length(punto) == length(mensaje) && length(punto) <= 1 
            x = parse(Float64, mensaje)^2
            sendMessage(tg, text = "$x", chat_id = msg.message.chat.id)
        end
    catch
        
    end
end

Para enviar fotos, videos, audios, etc, es básicamente lo mismo, pero tienen que en vez de poner text = "texto", se pone photo = foto, document = documento, video = video, etc... 

Por supuesto, no es obvio cómo generan cada uno de esos materiales. Por ahora ya saben generar gif animados, imágenes y csv's, que después pueden cargar usando open("nombre del archivo", "r"), es decir, foto = open("nombre del archivo", "r"), document = open("nombre del archivo", "r"), etc... Sobre el resto de tipo de archivos que podrían generar y cargar, necesitan otras paqueterías que no veremos en el curso (aunque existen, tanto para generar videos, como audios). Igual ya tienen bastante con esto. 

Ahora, podría pasar que no sólo quieran interactuar vía texto, sino vía imágenes, videos, etc. Para esto necesitan descargar el correspondiente archivo de internet. Telegram guarda los archivos por una hora en una página que les genera y que tiene la forma "https://api.telegram.org/file/bottoken/path". aquí bottoken significa bot+el token. Recuerden, el token es la llave para todo, en particular para encontrar los archivos de una conversación con un bot (entre particulares es más complejo porque no existe el token y los archivos están encriptados de punta a punta). 

Sólo les falta obtener el path. Para obtener el path se utiliza la función Telegram.API.getFille, que requiere del cliente (TelegramClient(token)), y el id del archivo. El id del archivo depende de qué tipo de archivo sea, pero en general es algo del estilo message.document.file_id o similar. 

Pueden hacer ejemplos para encontrar dónde está el id del archivo imprimiendo el mensaje completo y enviando distintos tipos de archivo. 

In [ ]:
token = ""
tg = TelegramClient(token)
run_bot() do msg
    @show msg
end

In [ ]:
using Images
token = ""
tg = TelegramClient(token)
path = Any[]
run_bot() do msg
    try 
        id_fille = msg.message.document.thumb.file_id
        path = [Telegram.API.getFile(tg, file_id =  id_fille)]
        @show path
    catch
        
    end
end

In [ ]:
using Images
token = ""
tg = TelegramClient(token)
path = Any[]
run_bot() do msg
    try 
        id_fille = msg.message.document.thumb.file_id
        path = Telegram.API.getFile(tg, file_id =  id_fille).file_path
        @show path
    catch
        
    end
end

In [ ]:
path = ""
run_bot() do msg
    try 
        id_fille = msg.message.document.file_id
        path = Telegram.API.getFile(tg, file_id =  id_fille).file_path
        imagen_pagina = "https://api.telegram.org/file/bot"*token*"/"*path
        @show imagen_pagina
    catch
        
    end
end

In [ ]:
imagen_pagina = ""
A = load(download(imagen_pagina))

In [ ]:
run_bot() do msg
    try 
        id_fille = msg.message.document.file_id
        path = Telegram.API.getFile(tg, file_id =  id_fille).file_path
        imagen_pagina = "https://api.telegram.org/file/bot"*token*"/"*path
        save("prueba1.png", load(download(imagen_pagina)))
        sendPhoto(photo = open("prueba1.png", "r"), chat_id = msg.message.chat.id)
    catch
        
    end
end

Y con esto ya tienen muchísimas posibilidades de bots que pueden crear. 

Por supuesto, siempre, para que puedan estar funcionando, necesitan tener un servidor donde esté corriendo sus programas, por ejemplo su computadora. Si apagan su computadora, el bot deja de funcionar. Pero eso es algo general de los bots, están corriendo en algún lugar y ese algún lugar es típicamente la compu del creador (o la compañía en la que trabaja). 

Y como en la mayoría de las paqueterías que veremos en este curso, esto es sólo una parte. Hay varias cosas más que se pueden hacer con esta paquetería, como tener botones de compra (pago de servicios o mercancías), aunque en realidad sí revisamos bastante de lo que se puede hacer. 